In [18]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import datetime
import seaborn as sns
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
import gc
import re
from collections import Counter
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing

In [19]:
feature_train=pd.read_pickle("./processed_data/df_train2.pkl")
feature_test=pd.read_pickle("./processed_data/df_test2.pkl")

In [20]:
feature_train.columns

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'label', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
       'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
       'truetime', 'day', 'hour', 'minute', '160_height', '160_width',
       '160_ppi', 'hw_ratio', 'inch'],
      dtype='object')

In [21]:
feature_test.columns

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
       'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
       'label', 'truetime', 'day', 'hour', 'minute', '160_height', '160_width',
       '160_ppi', 'hw_ratio', 'inch'],
      dtype='object')

In [61]:
cate_feature = ['apptype', 'carrier', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'location', 'fea1_hash', 'cus_type', 'day', 'hour', 'minute', '160_height', '160_width','160_ppi']
for item in cate_feature:
    le=preprocessing.LabelEncoder()
    feature_train[item] = le.fit_transform(feature_train[item])
    feature_test[item] = le.transform(feature_test[item])

prediction = feature_test[['apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
       'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type', 'day', 'hour', 'minute', '160_height', '160_width',
       '160_ppi', 'hw_ratio', 'inch']]


prediction['label'] = 0


y_col = 'label'
x_col=['apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package',
       'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type', 'day', 'hour', 'minute', '160_height', '160_width',
       '160_ppi', 'hw_ratio', 'inch']
# x_col = ['apptype', 'carrier', 'dev_height', 'dev_ppi','timestamp',
#        'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'version', 'fea_hash', 'location', 'fea1_hash', 'cus_type',
#        'Day', 'Hour', 'apptype_most_index', 'apptype_most_freq', 'carrier_most_index', 'carrier_most_freq',
#        'lan_most_index', 'lan_most_freq', 'media_id_most_index',
#        'media_id_most_freq', 'ntt_most_index', 'ntt_most_freq',
#        'os_most_index', 'os_most_freq', 'osv_most_index', 'osv_most_freq',
#        'package_most_index', 'package_most_freq', 'version_most_index',
#        'version_most_freq', 'location_most_index', 'location_most_freq',
#        'fea1_hash_most_index', 'fea1_hash_most_freq', 'cus_type_most_index',
#        'cus_type_most_freq', 'Day_most_index', 'Day_most_freq',
#        'Hour_most_index', 'Hour_most_freq']

model = lgb.LGBMClassifier(boosting_type='gbdt',num_leaves=128,max_depth=7,learning_rate=0.025,n_estimators=10000,subsample=0.6,feature_fraction=0.5,reg_alpha=1.2,reg_lambda=1.2,random_state=220,metric=None,n_jobs=-1,save_binary=True,cat_smooth=30
# ,max_bin=255
                        #    colsample_bytree=0.8
                        #    metric=["binary_error"]
                        #    max_bin=10,
                        #    device="gpu"
                           )

In [62]:
oof = []
df_importance_list = []
n=10
kfold = KFold(n_splits=n,shuffle=True,random_state=20)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(feature_train[x_col], feature_train[y_col])):
    X_train = feature_train.iloc[trn_idx][x_col]
    Y_train = feature_train.iloc[trn_idx][y_col]

    X_val = feature_train.iloc[val_idx][x_col]
    Y_val = feature_train.iloc[val_idx][y_col]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    lgb_model = model.fit(
        X_train,
        Y_train,
        categorical_feature=cate_feature,
        eval_names=['train', 'valid'],
        eval_set=[(X_train, Y_train), (X_val, Y_val)],
        verbose=100,
        eval_metric=["auc","binary_logloss","binary_error"],
        early_stopping_rounds=300
    )

    pred_val = lgb_model.predict_proba(X_val, num_iteration=lgb_model.best_iteration_)[:, 1]
    df_oof = feature_train.iloc[val_idx].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = lgb_model.predict_proba(feature_test[x_col], num_iteration=lgb_model.best_iteration_)[:, 1]
    prediction['label'] += pred_test / n

    df_importance = pd.DataFrame({
        'column': x_col,
        'importance': lgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

s auc: 0.958711	train's binary_logloss: 0.26046	train's binary_error: 0.100558	valid's auc: 0.943624	valid's binary_logloss: 0.291949	valid's binary_error: 0.11012
[900]	train's auc: 0.960152	train's binary_logloss: 0.256741	train's binary_error: 0.0991622	valid's auc: 0.943675	valid's binary_logloss: 0.291764	valid's binary_error: 0.1101
[1000]	train's auc: 0.961592	train's binary_logloss: 0.253013	train's binary_error: 0.0978422	valid's auc: 0.943721	valid's binary_logloss: 0.291632	valid's binary_error: 0.11
[1100]	train's auc: 0.963126	train's binary_logloss: 0.248963	train's binary_error: 0.0964022	valid's auc: 0.943726	valid's binary_logloss: 0.291615	valid's binary_error: 0.10982
Early stopping, best iteration is:
[870]	train's auc: 0.95974	train's binary_logloss: 0.257804	train's binary_error: 0.0995689	valid's auc: 0.943692	valid's binary_logloss: 0.291741	valid's binary_error: 0.10976

Fold_4 Training ================================

Training until validation scores don't im

In [60]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg('mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,minute,7948.4
1,hw_ratio,7942.1
2,timestamp,7547.0
3,fea_hash,7371.9
4,location,7044.1
5,cus_type,7006.1
6,dev_height,6436.4
7,fea1_hash,5847.9
8,hour,4996.5
9,inch,4506.8


In [49]:
a = pd.DataFrame(prediction.index)
a['label'] = lgb_model.predict(prediction[x_col])

In [17]:
# a.to_csv('submission_0609_0137.csv', index=False)

In [50]:
user_label=pd.DataFrame()
user_label["uid"]=feature_train.groupby("android_id").count().index
user_label=user_label.set_index("uid")

user_label["label_prior"]=feature_train.groupby("android_id").label.mean()

In [51]:
a["uid"]=feature_test.android_id.values
a=pd.merge(a,user_label,on="uid",how="left")

In [52]:
def post(label,prior):
    n=len(label)
    count=0
    for i in range(n):
        if 0<=prior[i]<=0.1 and label[i]==1:
            label[i]=0
            count+=1
            # print(i)
        elif 0.9<=prior[i]<=1 and label[i]==0:
            label[i]=1
            count+=1
            # print(i)
        else:
            pass
    print(count)
    return label.values

In [53]:
a.label=post(a.label,a.label_prior)

399


In [54]:
a=a[["sid","label"]]
a.to_csv('submission_0609_0206.csv', index=False)

In [55]:
a

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1


In [56]:
model

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               feature_fraction=0.4, importance_type='split',
               learning_rate=0.02, max_depth=6, metric=None,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=10000, n_jobs=-1, num_leaves=100, objective=None,
               random_state=220, reg_alpha=0.8, reg_lambda=0.8,
               save_binary=True, silent=True, subsample=0.6,
               subsample_for_bin=200000, subsample_freq=0)